## load dataset

In [1]:
from xml.etree.ElementTree import parse

In [2]:
def load_xml(file):
    docs_dict = {}
    doc_xml = parse(file)
    root = doc_xml.getroot()
    for item in root.findall('DOC'):
        docID = item.find('DOCNO').text
        headline = item.find('HEADLINE').text
        text = item.find('TEXT').text
        content = headline+text
        docs_dict[docID] = content
    return docs_dict

In [3]:
def load_stop_words(file):
    with open(file) as f:
        stop_words = [w.strip('\n') for w in f.readlines()]
    return stop_words

## Pre-processes text

In [4]:
import re

In [5]:
from nltk.stem import PorterStemmer

In [6]:
def preprocess(text):
    preprocessed_tokens = []
    stemmer = PorterStemmer()
    pattern = r"\w+"
    tokens = re.findall(pattern,text)
    for token in tokens:
        if token.lower() not in stop_words:
            preprocessed_tokens.append(stemmer.stem(token.lower()))
    return preprocessed_tokens

## Creates a positional inverted index

In [7]:
from collections import defaultdict, OrderedDict

In [8]:
def positional_inverted_index():
    index_dict = defaultdict(lambda:defaultdict(list))#  初始化一个双字典
    """
    {
    {doc_id:[1,2,3]},
    {doc_id:[1,2,3]},
    {doc_id:[2,3,4]}
    }
    
    """
    for docID,content in docs_dict.items():
        tokens = preprocess(content)
        for position,token in enumerate(tokens):
            index_dict[token][docID].append(position+1)
    ordered_index_dict = OrderedDict(sorted(index_dict.items()))#　排序
    return ordered_index_dict

In [9]:
def write_index_to_file(index):
    with open('index.txt', 'w') as f:
        for term in index.keys():
                line = term + ':' + str(len(index[term])) +'\n'
                for docID in index[term].keys():
                    position_list = index[term][docID]
                    line += '\t' + str(docID) + ': '+ ','.join(str(position) for position in position_list)+ '\n'
                f.write(line)            

## write positional inverted index to file

In [10]:
xml_path = "/home/congw/Projects/IR/CW1collection/trec.5000.xml"
stop_word_path = "/home/congw/Projects/IR/englishST.txt"
docs_dict = load_xml(xml_path)
stop_words = load_stop_words(stop_word_path)
index = positional_inverted_index()
write_index_to_file(index)

## Boolean search

In [11]:
def tokenize_query(query):
    """
    把query语句"合适的"拆开
    "Financial times" AND NOT BBC -> ["Financial times", AND, NOT, BBC]
    """
    query_list = query.split(' ')
    length = len(query_list)
    index = 0
    result_list = []
    
    while index < length:
        
        if '#' in query_list[index]:
            
            temp_list = []
            temp_list.append(query_list[index])
            while ')' not in query_list[index]:
                index+=1
                temp_list.append(query_list[index])
            index+=1
            result_list.append(''.join(temp_list))
        
        elif '\"' in query_list[index]:
            temp_list = []
            temp_list.append(query_list[index])
            index+=1
            while '\"' not in query_list[index]:
                temp_list.append(query_list[index])
                index+=1
            index+=1
            temp_list.append(query_list[index-1])
            result_list.append(' '.join(temp_list))
            
        else:
            result_list.append(query_list[index])
            index+=1    
    
    return result_list

In [12]:
def parse_proximity_query(query):
    """
    把　proximity query 用到的部分解析出来
    #15(dow,stocks) -> [15,dow,stocks]　（还要对每个query term 进行预处理）
    """
    proximity_parse = re.findall(r'#([0-9]+?)\((.+?)\)', query)
    max_distence = int(proximity_parse[0][0])
    query_terms = proximity_parse[0][1].split(',')
    preprocessed_query_terms = [PorterStemmer().stem(t.lower()) for t in query_terms]
    return preprocessed_query_terms,max_distence
def parse_phrasal_query(query):
    """
    同上
    """
    phrasal_parse = re.findall(r'\"(.+?)\"', query)
#     print(phrasal_parse)
    query_terms = phrasal_parse[0].split(' ')
    preprocessed_query_terms = [PorterStemmer().stem(t.lower()) for t in query_terms]
    return preprocessed_query_terms

In [13]:
def linear_merge(terms,max_dist,search="Phrasal"):
    """
    对query term 对应的posting list进行线性合并，
    如果是Phrasal query，查找是否有最大距离是１
    如果是proximity query，查找是否有最大距离是 max_dist
    """
    posting_lists = [index[term] for term in terms]
    docNums = [sorted(posting_list.keys()) for posting_list in posting_lists]
    result = []
    intersection_docNums = list(set(docNums[0]) & set(docNums[1]))
    for intersection_docNum in intersection_docNums:
        left_term_position = posting_lists[0][intersection_docNum]             
        right_term_position = posting_lists[1][intersection_docNum]                         
        abs_distences = [abs(j - i) for i in left_term_position for j in right_term_position]
        if search=="Phrasal":
            if max_dist in abs_distences:
                result.append(intersection_docNum)
        else:
            if any([abs_distence <= max_dist for abs_distence in abs_distences]):
                result.append(intersection_docNum)
    return result

In [14]:
def single_search(query):
    """
    处理单个 query term 查询
    """
    def is_find(words):
        flag = 1
        for word in words:
            if word not in index.keys():
                flag=0
        return flag
        
    if '\"' in query:
        query_terms = parse_phrasal_query(query)
        if is_find(query_terms):
            return linear_merge(query_terms,1,"Phrasal")
        else:
            return []
    
    elif '#' in query:
        query_terms,max_ditence = parse_proximity_query(query)
        if is_find(query_terms):
            return linear_merge(query_terms,max_ditence,"Proximity")
        else:
            return []
    else:
        query_term = [PorterStemmer().stem(query.lower())]
        if is_find(query_term):
            return sorted(index[query_term[0]].keys())
        else:
            return []
    

In [15]:
def bool_search(query):
    """
    处理布尔查询
    """
    query_lst = tokenize_query(query)
    key_words = ["AND","OR","NOT"]
    
    if len(query_lst) == 1:
        return set(single_search(query_lst[0]))
    
    if len(query_lst) == 3:
    
        if query_lst[1] == "AND":
            word1 = query_lst[0]
            word2 = query_lst[2]
            return set(single_search(word1)) & set(single_search(word2))

        else:
            word1 = query_lst[0]
            word2 = query_lst[2]
            return set(single_search(word1)) | set(single_search(word2))
    
    if len(query_lst) == 4:
        doc_ids = []     
        for term in index.keys():                               
            doc_ids.extend(index[term].keys())                     
        doc_ids = set(doc_ids)  
        
        if query_lst[1] == "AND":
            word1 = query_lst[0]
            word2 = query_lst[2]
            return set(single_search(word1)) & (doc_ids - set(single_search(word2)))
        
        else:
            word1 = query_lst[0]
            word2 = query_lst[2]
            return set(single_search(word1)) | (doc_ids - set(single_search(word2)))

In [16]:
def write_search_to_file(query_file):
    content = ""
    with open(query_file) as f:
        query_list = f.readlines()
    
    for query in query_list:
        pattern = r'([0-9]+?) (.+?)\n'
        query_id, query = re.findall(pattern, query)[0]
        print(query)
        result_list = list(bool_search(query))
        for doc_id in result_list:
            content += '{},{}\n'.format(query_id, doc_id)
                
    with open('results.boolean.txt', 'w') as f:         
        f.write(content)

In [17]:
write_search_to_file("/home/congw/Projects/IR/CW1collection/queries.boolean.txt")

Happiness
Edinburgh AND SCOTLAND
income AND taxes
"income taxes"
#20(income, taxes)
"middle east" AND peace
"islam religion"
"Financial times" AND NOT BBC
"wall street" AND "dow jones"
#15(dow,stocks)


## tf-idf

In [18]:
def count_unique_document():
    doc_ids = []
    for term in index.keys():                      
        doc_ids.extend(index[term].keys())                
    doc_ids = set(doc_ids)            
    return len(doc_ids)

N=count_unique_document()

In [19]:
import math

def weight(term,docID):
    tf = len(index[term][docID]) 
    df = len(index[term].keys())
    wt = 1 + math.log10(tf) * math.log10(N / df)
    return wt


In [20]:
def score(query, docID):
    score = 0
    query_terms = preprocess(query)
    for term in query_terms:
        if docID in index[term]:
            score += weight(term, docID)
    return score

In [21]:
def extract_docIDs(query):
    docIDs = []
    query_terms = preprocess(query)
    for term in query_terms:
        docIDs.extend(index[term].keys())
    return sorted(list(set(docIDs)))

In [22]:
def rank(query):
    scores = dict()
    for docID in extract_docIDs(query):          
        scores[docID] = score(query, docID)
    return {k: v for k, v in sorted(scores.items(), key=lambda item: item[1],reverse=True)}

In [23]:
def write_ranked_query_to_file(query_file,MAX=150):
    
    def load_query_file(query_file):
        with open(query_file) as f:
            queries = f.readlines()
        return queries
    def write_ranked_result(content,file="results.ranked.txt"):
        with open(file,"w") as f:
            f.write(content)
    
    queries = load_query_file(query_file)
    pattern = r'([0-9]+?) (.+?)\n' 
    content = ""
    
    for query in queries:
        queryID, query = re.findall(pattern, query)[0]
        ranked_dict = rank(query)
        count = 0
        for docID,score in ranked_dict.items():
            if count < MAX:
                content += "{},{},{:.4f}\n".format(queryID, docID, score)
                count+=1
        
    write_ranked_result(content)

In [24]:
write_ranked_query_to_file("/home/congw/Projects/IR/CW1collection/queries.ranked.txt")